Ваша задача — реализовать алгоритм случайного леса для задачи классификации и применить его к набору данных.

Шаги выполнения задания:

1. Импортируйте необходимые библиотеки: numpy, pandas, sklearn.

In [261]:
import numpy as np
import pandas as pd
import matplotlib as  plt   
import seaborn as  sns
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score



2. Загрузите набор данных для задачи классификации. Вы можете использовать любой доступный набор данных или выбрать один из популярных, таких как Iris, Wine или MNIST.

In [262]:
data = load_wine()

In [263]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
dtypes: fl

3. Проведите предварительную обработку данных, включая масштабирование и разделение на обучающую и тестовую выборки.

In [264]:
X_train, X_test, y_train, y_test = train_test_split(df, y\
                                                    , train_size=0.8\
                                                    , random_state=42
                                                    , stratify=y)
print(f'{[el.shape for el in [X_train, X_test, y_train, y_test]]}')

[(142, 13), (36, 13), (142,), (36,)]


4. Реализуйте алгоритм случайного леса. Включите в него функции для построения деревьев решений, выбора случайных признаков и голосования для принятия решений.

In [265]:
class RandomForestClassifierCustom():
    def __init__(self, criterion="gini", n_estimators=100, max_depth=None, random_state=None, max_samples=None):
        self.criterion = criterion
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.max_samples = max_samples
        self.estimators = []

    def fit(self, X, y):
        if self.max_samples is None:
            max_samples = X.shape[0]
        elif isinstance(self.max_samples, float):
            if 0 < self.max_samples <= 1:
                max_samples = round(X.shape[0] * self.max_samples)
            elif self.max_samples <= 0:
                raise ValueError(f"max_samples can't be less or equal zero")
        else:
            max_samples = self.max_samples
        print(max_samples)
        rng = np.random.default_rng(self.random_state)
        #  Сформируем набор для инициалиации random_state для каждого цикла
        random_state_list = rng.choice(100, self.n_estimators)
        for i in range(self.n_estimators):
            rng = np.random.default_rng(random_state_list[i])
            indices = rng.choice(X.shape[0], max_samples)
            # print(indices)
            X_subset = X.iloc[indices]
            y_subset = y[indices]
            model = DecisionTreeClassifier(criterion=self.criterion, max_depth=self.max_depth, random_state=self.random_state)
            model.fit(X_subset, y_subset)
            self.estimators.append(model)            

    def predict(self, X):
        y_preds = []
        for i in range(len(X)):
            votes = {}  # Словарь для подсчета голосов деревьев за каждый класс
            for clf in self.estimators: # предсказание для отдельного наблюдения
                pred = clf.predict(X.iloc[[i]])[0]  # Предсказание метки на текущем дереве для отдельного наблюдения 
                if pred not in votes:
                    votes[pred] = 1
                else:
                    votes[pred] += 1
            y_preds.append(max(votes, key=votes.get))  # Выбор метки с наибольшим количеством голосов
        return np.array(y_preds) 


5. Обучите вашу модель случайного леса на обучающей выборке.

In [266]:
m = RandomForestClassifierCustom(n_estimators=100, max_samples=0.5)
m.fit(X_train, y_train)
y_pred = m.predict(X_test)

71


6. Оцените производительность модели на тестовой выборке, используя метрики классификации, такие как точность, полнота и F1-мера.

In [267]:
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='macro'))

# Наблюдаем отличные результаты работы модели!!!

1.0
1.0
1.0


7. Проведите сравнение результатов вашей модели со стандартной реализацией случайного леса из библиотеки scikit-learn.

In [268]:
model_rfc = RandomForestClassifier(n_estimators=100, max_samples=0.5)
model_rfc.fit(X_train, y_train)
y_pred = model_rfc.predict(X_test)

In [269]:
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='macro'))

# Наблюдаем такие же отличные результаты работы модели.

1.0
1.0
1.0


In [270]:
# Попробуем модель на более сложном датасете
df1 = pd.read_csv('../S8/telecom_churn.csv')

In [271]:
df1.info()
# Пропусков нет, можно работать!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   3333 non-null   object 
 1   Account length          3333 non-null   int64  
 2   Area code               3333 non-null   int64  
 3   International plan      3333 non-null   object 
 4   Voice mail plan         3333 non-null   object 
 5   Number vmail messages   3333 non-null   int64  
 6   Total day minutes       3333 non-null   float64
 7   Total day calls         3333 non-null   int64  
 8   Total day charge        3333 non-null   float64
 9   Total eve minutes       3333 non-null   float64
 10  Total eve calls         3333 non-null   int64  
 11  Total eve charge        3333 non-null   float64
 12  Total night minutes     3333 non-null   float64
 13  Total night calls       3333 non-null   int64  
 14  Total night charge      3333 non-null   

In [272]:
if 'State' in df1.columns:
    df1.drop(columns=['State'], inplace=True)
df1['Voice mail plan'] = df1['Voice mail plan'].map({'Yes':1, 'No':0})
df1['International plan'] = df1['International plan'].map({'Yes':1, 'No':0})#.value_counts()
df1['Churn'] = df1['Churn'].map({True:1, False:0})

In [273]:
df1

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,0,1,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,0
3329,68,415,0,0,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,0
3330,28,510,0,0,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,0
3331,184,510,1,0,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2,0


In [274]:
X_train, X_test, y_train, y_test = train_test_split(df1.drop(columns='Churn'), df1['Churn'].values\
                                                    , train_size=0.7\
                                                    , random_state=42
                                                    , stratify=df1['Churn'])
print(f'{[el.shape for el in [X_train, X_test, y_train, y_test]]}')

[(2333, 18), (1000, 18), (2333,), (1000,)]


In [275]:
m = RandomForestClassifierCustom(n_estimators=100, max_samples=0.5, random_state=42)
m.fit(X_train, y_train)
y_pred = m.predict(X_test)

1166


In [276]:
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(roc_auc_score(y_test,y_pred))

# Метрики получились лучше, чем метрики одного решающего дерева, для аналогичной задачи в прошлом ДЗ.

0.7448275862068966
0.8150943396226416
0.8653962492437992


In [277]:
model_rfc = RandomForestClassifier(n_estimators=100, max_samples=0.5, random_state=42)
model_rfc.fit(X_train, y_train)
y_pred = model_rfc.predict(X_test)

print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(roc_auc_score(y_test,y_pred))

# Странно, что метрики получились похуже, чем у самостоятельно реализованной модели.
# Но при этом скорость работы модели из библиотеки sklearn значительно выше, чем у самостоятельно реализованной.

0.696551724137931
0.7984189723320159
0.8441822948175035
